# Домашнее задание: декораторы

## Импорт библиотек, установка констант

In [12]:
import requests
import time
import re

from random import randint
from functools import wraps
from time import perf_counter

BOOK_PATH = 'https://www.gutenberg.org/files/2638/2638-0.txt'

## Задание 1
Реализуйте декоратор ```benchmark(func)```, выводящий время, которое заняло выполнение декорируемой функции

In [13]:


def benchmark(func):
    """
    Декоратор, выводящий время  в секундах, которое заняло выполнение декорируемой функции
    """

    @wraps(func)
    def wrapper(*args, **kwargs):
        time_start = perf_counter()
        result = func(*args, **kwargs)
        time_end = perf_counter()
        time_duration = time_end - time_start
        print(f'Время выполнения функции {func.__name__} {time_duration:.6f} секунд')
        return result
    return wrapper


## Задание 2
Реализуйте декоратор ```logging(func)```, который выводит аргументы с которыми была вызвана функция

In [14]:

def logging(func):
    """
    Декоратор, который выводит аргументы с которыми была вызвана функция
    """

    @wraps(func)
    def wrapper(*args, **kwargs):
        print("Функция вызвана с аргументами:")
        if args:
          print("Позиционные:")
          for _ in args:
            print(_)
        if kwargs:
          print("Именованные:")
          for key, value in kwargs.items():
            print(f"{key}: {value}")
        if not args and not kwargs:
            print("Аргументы отсутствуют")
        result = func(*args, **kwargs)
        return result
    return wrapper


## Задание 3
Реализуйте декоратор ```counter(func)```, считающий и выводящий количество вызовов декорируемой функции  

---

Декоратор реализован таким образом, что сохраняет количество вызовов каждой функции отдельно. Для обнуления количества вызовов конкретной функции реализована функция ```counter_truncate(func)```.

In [15]:
def counter(func):
    """
    Декоратор, считающий и выводящий название и количество вызовов декорируемой функции
    """
    global all_counters
    all_counters = {}

    @wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        if func.__name__ in all_counters:
            before = all_counters[func.__name__]
            all_counters[func.__name__] = before + 1
        if func.__name__ not in all_counters:
            all_counters[func.__name__] = 1
        print(f"Функция {func.__name__} была вызвана: {all_counters[func.__name__]} раз(a)")
        return result
    return wrapper


def counter_truncate(func):
    """
    Функция, обнуляющая количество вызовов декорируемой функции
    """
    all_counters[func.__name__] = 0


# Пример
# Объявим 2 функции:
@counter
def first():
    pass

@counter
def second():
    pass

# Вызовем их:
first()

for _ in range(3):
    second()

# Обнулим функцию second:
counter_truncate(second)

# Вызовем их повторно:
first()

for _ in range(3):
    second()


Функция first была вызвана: 1 раз(a)
Функция second была вызвана: 1 раз(a)
Функция second была вызвана: 2 раз(a)
Функция second была вызвана: 3 раз(a)
Функция first была вызвана: 2 раз(a)
Функция second была вызвана: 1 раз(a)
Функция second была вызвана: 2 раз(a)
Функция second была вызвана: 3 раз(a)


## Задание 4  
Реализуйте декоратор ```memo(func)```, запоминающий результаты исполнения функции ```func```, чьи аргументы ```*args``` должны быть хешируемыми. Сравните время выполнения рекурсивной реализации расчета чисел Фибоначчи без декоратора и с ним.  

---

В рамках этого задания был реализован класс ```LRUCache``` (собственная реализация LRU кэша на базе словаря). Декоратор ```memo``` проверяет на хэшируемость все аргументы передаваемой в него функции. Если все аргументы хэшируемы - результат выполнения функции кэшируется. Если хоть один аргумент не хэшируем - результат выполнения функции не кэшируется.

In [16]:
class LRUCache:

    def __init__(self, limit: int = 124):
        self.limit = limit
        self.dict_cache = {}
        if not isinstance(limit, int):
            raise ValueError("limit must be an integer")
        if limit < 0:
            raise ValueError("limit must be a positive number")

    def get(self, key):
        if key in self.dict_cache:
            value = self.dict_cache.pop(key)
            self.dict_cache[key] = value
            return value
        return None

    def set(self, key, value):
        if len(self.dict_cache.items()) < self.limit:
            if key not in self.dict_cache:
                self.dict_cache[key] = value
            elif key in self.dict_cache:
                self.dict_cache.pop(key)
                self.dict_cache[key] = value
        else:
            if self.limit == 0:
                pass
            elif key not in self.dict_cache:
                self.dict_cache.pop(next(iter(self.dict_cache)))
                self.dict_cache[key] = value
            elif key in self.dict_cache:
                self.dict_cache.pop(key)
                self.dict_cache[key] = value


def memo(func):
    """
    Декоратор, запоминающий результаты исполнения функции func, чьи аргументы args должны быть хешируемыми
    """
    lrucache = LRUCache()

    @wraps(func)
    def memo_wrapper(*args):
        try:
            [hash(arg) for arg in args]
            if lrucache.get(str(args)):
                result = lrucache.get(str(args))
            else:
                result = func(*args)
                lrucache.set(str(args), result)
            return result
        except TypeError:
            result = func(*args)
            return result
    return memo_wrapper


## Тестирование

In [17]:
@counter
@logging
@benchmark
def word_count(word, url=BOOK_PATH):
    """
    Функция для посчета указанного слова на html-странице
    """

    # отправляем запрос в библиотеку Gutenberg и забираем текст
    raw = requests.get(url).text

    # заменяем в тексте все небуквенные символы на пробелы
    processed_book = re.sub('[\W]+' , ' ', raw).lower()

    # считаем
    cnt = len(re.findall(word.lower(), processed_book))

    return f"Cлово {word} встречается {cnt} раз"

print(word_count('whole'))

Функция вызвана с аргументами:
Позиционные:
whole
Время выполнения функции word_count 0.602267 секунд
Функция word_count была вызвана: 1 раз(a)
Cлово whole встречается 176 раз


In [54]:
@benchmark
def fib(n):
    if n < 2:
        return n
    return fib(n-2) + fib(n-1)

# измеряем время выполнения с помощью @benchmark (без кэширования)
fib(5)

Время выполнения функции fib 0.000001 секунд
Время выполнения функции fib 0.000001 секунд
Время выполнения функции fib 0.000001 секунд
Время выполнения функции fib 0.000906 секунд
Время выполнения функции fib 0.002421 секунд
Время выполнения функции fib 0.000001 секунд
Время выполнения функции fib 0.000001 секунд
Время выполнения функции fib 0.001738 секунд
Время выполнения функции fib 0.000001 секунд
Время выполнения функции fib 0.000001 секунд
Время выполнения функции fib 0.000000 секунд
Время выполнения функции fib 0.000035 секунд
Время выполнения функции fib 0.000082 секунд
Время выполнения функции fib 0.002098 секунд
Время выполнения функции fib 0.004994 секунд


5

In [66]:
def fib(n):
    if n < 2:
        return n
    return fib(n-2) + fib(n-1)

In [67]:
# измеряем время выполнения с помощью %%time (без кэширования)
%%time
fib(20)

CPU times: user 4.31 ms, sys: 0 ns, total: 4.31 ms
Wall time: 5.71 ms


6765

In [58]:
@memo
@benchmark
def fib(n):
    if n < 2:
        return n
    return fib(n-2) + fib(n-1)

# измеряем время выполнения с помощью @benchmark (c кэшированием)
fib(5)

Время выполнения функции fib 0.000001 секунд
Время выполнения функции fib 0.000001 секунд
Время выполнения функции fib 0.000050 секунд
Время выполнения функции fib 0.001714 секунд
Время выполнения функции fib 0.000007 секунд
Время выполнения функции fib 0.001767 секунд


5

In [68]:
@memo
def fib(n):
    if n < 2:
        return n
    return fib(n-2) + fib(n-1)

In [69]:
# измеряем время выполнения с помощью %%time (c кэшированием)
%%time
fib(20)

CPU times: user 171 µs, sys: 0 ns, total: 171 µs
Wall time: 175 µs


6765